In [1]:
import csv 
import os
import numpy as np
import pandas as pd
import ants
import seaborn as sns
import dill as pickle 
import shutil
import glob

from tqdm import tqdm
from copy import deepcopy

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

In [3]:
MOUSE_path = f'{os.environ["HOME"]}/mouse_dataset'
NEW_MOUSE_path = f'{os.environ["HOME"]}/new_mouse_dataset'

os.system(f'mkdir -p {MOUSE_path}/yale_to_orig_template')

0

In [4]:
# yale inputs
ytemp_ras_100_img = ants.image_read(
    f'{MOUSE_path}/gabe_symmetric_N162/Symmetric_N162_0.10.nii.gz'
)
ytemp_lpi_100_img = ants.image_read(
    f'{MOUSE_path}/gabe_symmetric_N162/Symmetric_N162_0.10_RAS.nii.gz'
)
ytemp_ras_200_img = ants.image_read(
    f'{MOUSE_path}/gabe_symmetric_N162/Symmetric_N162_0.20.nii.gz'
)
ytemp_lpi_200_img = ants.image_read(
    f'{MOUSE_path}/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
)
display(ytemp_lpi_100_img)

dtemp_lpi_100_img = ants.image_read(
    f'{MOUSE_path}/gabe_resampled_DSURQE/DSURQE_average_to_N162_RAS_masked.nii.gz'
)
dtemp_lpi_100_mask_img = ants.image_read(
    f'{MOUSE_path}/gabe_resampled_DSURQE/DSURQE_mask_to_N162_RAS.nii.gz'
)
display(dtemp_lpi_100_img)
display(dtemp_lpi_100_mask_img)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [5]:
# original inputs
otemp_lpi_40_img = ants.image_read(
    f'{MOUSE_path}/mouse-brain-templates-0.5.3/dsurqec_40micron_masked.nii'
)
otemp_lpi_100_img = ants.resample_image(
    otemp_lpi_40_img, 
    resample_params=[0.1, 0.1, 0.1],
)
display(otemp_lpi_100_img)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (126, 191, 96)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [6]:
# linear registration
os.system('export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1')
os.system('export ANTS_RANDOM_SEED=1')

tx_ln = ants.registration(
    fixed=otemp_lpi_100_img,
    moving=dtemp_lpi_100_img,
    type_of_transform='TRSAA',
    random_seed=args.SEED,
)

for idx, tx in enumerate(tx_ln['fwdtransforms']):
    endswith = '.'.join(tx.split('.')[1:])
    shutil.copy2(tx, f'{MOUSE_path}/yale_to_orig_template/transformation_lin{idx:02d}.{endswith}')

In [7]:
dtemp_lpi_100_tx_ln_img = ants.apply_transforms(
    fixed=otemp_lpi_100_img,
    moving=dtemp_lpi_100_img,
    transformlist=tx_ln['fwdtransforms'],
    interpolator='linear',
)
dtemp_lpi_100_tx_ln_img.to_file(f'{MOUSE_path}/yale_to_orig_template/yale_template_tx_lin.nii.gz')
dtemp_lpi_100_tx_ln_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (126, 191, 96)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [8]:
# non linear transformation
os.system('export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1')
os.system('export ANTS_RANDOM_SEED=1')

tx_nl = ants.registration(
    fixed=otemp_lpi_100_img,
    moving=dtemp_lpi_100_img,
    type_of_transform='SyN',
    random_seed=args.SEED,
)

for idx, tx in enumerate(tx_nl['fwdtransforms']):
    endswith = '.'.join(tx.split('.')[1:])
    shutil.copy2(tx, f'{MOUSE_path}/yale_to_orig_template/transformation_nonlin{idx:02d}.{endswith}')

In [9]:
dtemp_lpi_100_tx_nl_img = ants.apply_transforms(
    fixed=otemp_lpi_100_img,
    moving=dtemp_lpi_100_img,
    transformlist=tx_nl['fwdtransforms'],
    interpolator='linear',
)
dtemp_lpi_100_tx_nl_img.to_file(f'{MOUSE_path}/yale_to_orig_template/yale_template_tx_nonlin.nii.gz')
dtemp_lpi_100_tx_nl_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (126, 191, 96)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [15]:
# apply dtemp mask to ytemp images and transform them to orig space
ytemp_lpi_100__masked_img = ants.mask_image(
    image=ytemp_lpi_100_img,
    mask=dtemp_lpi_100_mask_img,
    binarize=False,
)

# load transform
tx_list = sorted(glob.glob(f'{MOUSE_path}/yale_to_orig_template/transformation_nonlin*', recursive=True))
display(tx_list)

# apply transform and save the image
ytemp_lpi_100__masked_tx_img = ants.apply_transforms(
    fixed=otemp_lpi_100_img,
    moving=ytemp_lpi_100__masked_img,
    transformlist=tx_list,
    interpolator='linear',
)
ytemp_lpi_100__masked_tx_img.to_file(f'{MOUSE_path}/yale_to_orig_template/Symmetric_N162_0.10_RAS_masked_tx_nonlin.nii.gz')
display(ytemp_lpi_100__masked_tx_img)

['/home/govindas/mouse_dataset/yale_to_orig_template/transformation_nonlin00.nii.gz',
 '/home/govindas/mouse_dataset/yale_to_orig_template/transformation_nonlin01.mat']

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (126, 191, 96)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]